# Async JavaScript

## **1. Demonstrate JavaScript's Single-Threaded Nature**
question:

write an example to show that JavaScript is **single-threaded** by creating two competing tasks, one that
blocks the event loop and another async function that waits for a promise.


=>

Javascript is single-threaded. A thread is a sequence of instructions that a program follows, it can only do one thing at a time: so if it is waiting for our long-running synchronous call to return, it can't do anything else.

In [ ]:
%%javascript
console.log("Program start");

function asyncTask() {
  console.log("Promise start");
  return Promise.resolve("Promise done");
}

asyncTask().then((result) => console.log(result));

function blockingTask() {
  console.log("blocking task start");
  for (let i = 0; i < 10000000000; i++) {
    // do some task
  }
  console.log("blocking task end");
}

blockingTask();
console.log("Program end");

/* output:
 Program start
 Promise start
 blocking task start
 blocking task end
 Program end
 Promise done
*/

<IPython.core.display.Javascript object>

Above code snippet proves that Javascript is a **single-threaded.**

It means, in the javascript runtime engine there is one main execution context called **callstack.** Where all synchronous code is executed line by line first, execute **synchronous** code only.

Asynchronous operations (like setTimeout, fetch, Promises, etc.) do not directly go on the call stack. Instead:

The asynchronous operation is handled by the Web APIs or Node APIs provided by the environment. Once the asynchronous operation completes, its callback (or promise .then/.catch) is sent to the **callback queue** (or microtask queue for promises). The **event loop** checks if the call stack is empty, and if so, it pushes the next callback from the queue onto the call stack for execution.

**Step by step example execution**

1. `console.log("Program start")`  
* logs message in the javascript console `Program start`.
2. Function `asyncTask` called
* inside `asyncTask` function logs message `Promise start`.
* returns `Promise.resolve("Promise done")`.
* `Promise.resolve()` creates a resolved promise immediately, but the `.then()` callback is scheduled to run in the microtask queue, after the current synchronous code finishes.
3. Function `blockingTaksk` is called
* logs `blocking task start`
* this function defines heavy loop, this loop blocks the **event loop**. Because this loop might take long time to finish.
* after finish the loop logs `blocking task end`
* `console.log("Program end")`.

* and the last `Promise done` message prints, because in the runtime **event loop** sees that all the synchronous code execution done, then start taking  function from the microtask queue and execute that function in the **call stack**.

<img src="https://drive.google.com/uc?export=view&id=19-kTXqAS-h_5td_Ed7BrQo18tLKeIquy" width="100%"  alt="module-09-assignment-task-04"/>






## **2. Why Does JavaScript Not Execute Asynchronously by Default?**

question:

JavaScript is often called **synchronous** and **single-threaded**, yet it handles asynchronous tasks like AJAX requests, timers, and event listeners

a. Explain why JavaScript does not execute asynchronously by default.

b. Write a code snippet to prove that JavaScript is inherently synchronous.

=>

Javascript is **synchronous** which means code execution happen line by line, each line of code waits for previous line to finish. **single-threaded** means, it can only do one thing at a time.

The question is that, why does JS not execute async by default?

Assume that, JS exceute asychronous code by defalut, and suppose an application needs get some data from server. The data fetching task might take some time. If the data fetching/AJAX request async task execute in the global execution thread and JS is **single-threaded**. Then code execution will block here and user have to wait until finish the task. That will bring bad user experience.

There are other reasons too, but blocking global code execution thread is one biggest reason why JS is not execute async code by default. Althogh JS runtime engine can perform async code in superb way(Promise, setTimeout, setInterval).

In [ ]:
%%javascript

console.log("start");

for(let i = 0; i < 10000000000; i++) {
}

console.log("end")

<IPython.core.display.Javascript object>

The above code example execute synchronously. In the `for loop` completing the code execution might take time, while executing this code JS will not do anything else. It finish the present line of code then go to the next line.

## 3.Chaining Promises with setTimeout, modify the delay function to chain multiple promises so that three messages are logged in sequence with delays

In [ ]:
%%javascript

function delay(seconds) {
  return new Promise((resolve) => {
    setTimeout(resolve, seconds * 1000);
  });
}

delay(1)
  .then(() => {
    console.log("waited 1 sec");
    return delay(2);
  })
  .then(() => {
    console.log("waited 2 sec");
    return delay(3);
  })
  .then(() => console.log("waited 3 sec"));

## 4. What are the different states of a Promise, and how do they transition?

=>

In Javascript Promise is an object that is used as placeholder for the future result of an asynchronous operations. In promise there are three main states:
* Pending
* Fulfilled
* Rejected

**Pending:**
This is the initial state of a promise, before the future value is available. Durring this time the asychronous task is still doing its work in the background, meaning it can be fulfilled or rejected.

**Fullfiled:**
A fulfilled promise, is promise that has successfully resulted in a value just as we expected. For example, when we use a promise to fetch data from an API, a fulfilled promise successfully got that data, and it's available to being used.

**Rejected:**
A rejected promise means that there has been an error durring the asynchronous task. In the example of fetching data from an API, an error would be, for example, when the user is offline, and can't connect to the API server.


## 5. How does the JavaScript event loop handle Promises differently from setTimeout?

Javascript is **single-threaded** and sychronous by default. To handle asynchronous operations, Javascript runtime engine has two primary queues:
1. **Callback queue:** This queue stores tasks that are scheduled by asynchronous operations like `setTimeout`, `setInterval` and user interactions. Tasks in this queue are executed after the call stack is empty

2. **Microtask queue:** This queue holds tasks scheduled by promises and other microtasks. Microtasks have higher priority than tasks in the Callback Queue and are executed before the timer event.

In [ ]:
%%js
console.log("Start");

setTimeout(() => {
  console.log("timer finish");
}, 0);

new Promise((res, rej) => {
  for (let i = 0; i < 10000000000; i++) {
    // do some task
  }
  res("promise done");
}).then((res) => console.log(res));

console.log("end");

/* Ouput:
Start
end
promise done
timer finish
*/


If we see output of the above code example, then one question will arise that is, why `promise done` logged before `timer done`. `setTimeout` and `Promise` both are for performing asynchronous task, in the above code example `setTimeout` function received 0 second timer and it should call immediately, but it calls after the `promise done`. Why?

To understand the logic behind lets break down the code example-

Javascript will execute synchronous code first in the call stack. In this case two console.log `start` and `end` will execute first, then Javascript sees that there are two async operations `setTimeout` and `promise`. Javascript will handed over these two async operation to runtime engine. And runtime engine perform these async operations in the background. If async task is done, then its callback placed in the queue.

In this case setTimeout will placed in the `Callback queue` and callback returned by promise will placed in the `Microtask queue`.

Event loop constantly checks if the call stack empty or not, if its empty and no synchronous code is left, the event loop start taking callback from the queue. It will start taking callback from `Microtask queue` first and then `Callback queue`.

That is the reason why `promise done` logs before `timer finish`.

<img src="https://drive.google.com/uc?export=view&id=10EmCWphbAFOORcHXMO4atEEvRsmX_roD" width="100%"  alt="Asynchronous javascript"/>

